# The Farmer, Wolf, Cabbage, and Sheep Puzzle

This is a well known puzzle. A farmer is on the side of the river with his wolf, cabbage, and sheep. On the bank of the river is a small boat which can hold the farmer and only one more item (e.g., sheep, wolf, or cabbage). 

If the farmer leaves the wolf and the sheep on one side of the river alone, then the wolf will eat the sheep. Similarly, the sheep will eat the cabbage if the farmer is not there to stop it.

Help the farmer reach the other side of the river with his wolf, sheep, and cabbage intact.

## Representation

First let us consider this problem from an Artificial Intelligence perspective. How do we represent the state space? What are the actions that we can perform in this domain?

### State Space

The state space representation must encapsulate all important information about the domain. In this case, the only thing that matters what side of the river the various items are on.

So we may consider a simple dictionary with items and which side of the river they are on.

```
state = { 'wolf': 'left', 'sheep':'right', 'cabbage':'left', 'farmer':'left' }
```

In [12]:
initial = { 'wolf': 'left', 'sheep':'left', 'cabbage':'left', 'farmer':'left' }

### IsGoal Predicate

We need to write a function that checks if we have found a solution. In this case, we need to check if wolf, goat, sheep, and farmer are all on the right side of the river.

In [13]:
def isGoal( state ):
    return ( ( state['farmer'] == 'right') 
            and ( state['wolf'] == 'right') 
            and ( state['cabbage'] == 'right')  
            and ( state['sheep'] == 'right') )

### Sucessor Function

Now I define a function that generates all possible successors (children) for a given state. What I need to record is the children state, but also the action that I applied to reach this state.

So I define a successor function as well as a couple of utility functions.

def otherSide( side ):
    other = '?'
    if ( side == 'left' ):
        other = 'right'
    elif ( side == 'right' ):
        other = 'left'
    return other
        
def successors( state ):
    farmerSide=state['farmer']
    children = []
    for k in state:
        if ( state[k] == farmerSide ):
            newState = state.copy()
            newState[k] = otherSide( farmerSide )
            newState['farmer'] = otherSide( farmerSide )
            children = children + [ (newState, "move " + k + " to " + otherSide( farmerSide ) ) ]
    return children

print( 'Successors of state', initial, 'are', successors( initial ) )
s = { 'wolf': 'left', 'sheep':'right', 'cabbage':'left', 'farmer':'right' }
print( 'Successors of state', s, 'are', successors( s ) )

### Checking for invalid States

The successor function so far does not check if a state is illegal. So we need to add a check for invalid states (e.g., wolf and sheep on left side and farmer is on the right side).

In [29]:
def isValidState( state ):
    valid = True
    checkSide = otherSide( state['farmer'] )
    if ( state['wolf'] == checkSide ) and ( state['sheep'] == checkSide ):
        valid = False
    elif ( state['sheep'] == checkSide ) and ( state['cabbage'] == checkSide ):
        valid = False
    return valid

def successors( state ):
    farmerSide=state['farmer']
    children = []
    for k in state:
        if ( state[k] == farmerSide ):
            newState = state.copy()
            newState[k] = otherSide( farmerSide )
            newState['farmer'] = otherSide( farmerSide )
            if ( isValidState( newState ) ):
                children = children + [ (newState, "move " + k + " to " + otherSide( farmerSide ) ) ]
    return children

print( 'Successors of state', initial, 'are', successors( initial ) )
s = { 'wolf': 'left', 'sheep':'right', 'cabbage':'left', 'farmer':'right' }
print( 'Successors of state', s, 'are', successors( s ) )


Successors of state {'wolf': 'left', 'sheep': 'left', 'cabbage': 'left', 'farmer': 'left'} are [({'wolf': 'left', 'sheep': 'right', 'cabbage': 'left', 'farmer': 'right'}, 'move sheep to right')]
Successors of state {'wolf': 'left', 'sheep': 'right', 'cabbage': 'left', 'farmer': 'right'} are [({'wolf': 'left', 'sheep': 'left', 'cabbage': 'left', 'farmer': 'left'}, 'move sheep to left'), ({'wolf': 'left', 'sheep': 'right', 'cabbage': 'left', 'farmer': 'left'}, 'move farmer to left')]


## Search

Now it is time to implement a search routine for this domain. We will implement three methods

    1.) Depth First Search
    2.) Breadth First Search
    3.) Iterative Deepening Depth First Search
    
### Depth First Search

Depth first search starts at the initial state, and calculates the successors (i.e., expands the node). Then it will pick the first child and expand it. Then the process will continue recursively by expanding the first child. 

If there are no more children left for a node, then the search will backtrack and expand the next child on the level above.

Depth First Search lends itself to a recursive implementation, but to make it more conistent with the other search methods, I use a iterative approach with a queue. 

I am cheating a little bit here because the we will expand the last child generated first, rather than the first child.

I also include a depth limit, which will come in handy when we are dealing with iterative deepening depth first search.

In [31]:
def DFsearch( initial, depthLimit = 0, log = False ):
    queue = [ ( initial, 0, [initial], ['*** initial ***'] ) ]
    result = None
    numNodes = 1
    while( len(queue) > 0 ):
        state, level, history, actions = queue.pop()
        if ( depthLimit <= 0 ) or ( level < depthLimit ):
            if ( isGoal( state ) ):
                result = ( state, history, actions + [ '*** goal ***' ], numNodes )
                break
            children = successors( state )
            for ns, na in children:
                if ( ns not in history ):
                    node = ( ns, level+1, history + [ ns], actions + [na] )
                    queue.insert( len(queue), node )
                    numNodes = numNodes + 1
    if result is None:
        result = [ numNodes ]
    return result


In [32]:
DFsearch( initial, -1, False )

({'cabbage': 'right', 'farmer': 'right', 'sheep': 'right', 'wolf': 'right'},
 [{'cabbage': 'left', 'farmer': 'left', 'sheep': 'left', 'wolf': 'left'},
  {'cabbage': 'left', 'farmer': 'right', 'sheep': 'right', 'wolf': 'left'},
  {'cabbage': 'left', 'farmer': 'left', 'sheep': 'right', 'wolf': 'left'},
  {'cabbage': 'right', 'farmer': 'right', 'sheep': 'right', 'wolf': 'left'},
  {'cabbage': 'right', 'farmer': 'left', 'sheep': 'left', 'wolf': 'left'},
  {'cabbage': 'right', 'farmer': 'right', 'sheep': 'left', 'wolf': 'right'},
  {'cabbage': 'right', 'farmer': 'left', 'sheep': 'left', 'wolf': 'right'},
  {'cabbage': 'right', 'farmer': 'right', 'sheep': 'right', 'wolf': 'right'}],
 ['*** initial ***',
  'move sheep to right',
  'move farmer to left',
  'move cabbage to right',
  'move sheep to left',
  'move wolf to right',
  'move farmer to left',
  'move sheep to right',
  '*** goal ***'],
 10)

In [33]:
DFsearch( initial, 4, False )

[7]

### Iterative Deepening Depth First Search

Since my `DFsearch` routine alreadys allows the use of a depth limit, it is easy to implement a wrapper that will manage the depth limit and raises it, if no solution is found.

In [42]:
def IDDFsearch( initial, maxLevel = 1000, log = False):
    numNodes = 0
    result = None
    for level in range( 1, maxLevel ):
        result = DFsearch( initial, level, log )
        if ( log ):
            print( result )
        if ( len( result ) == 4 ):
            goal, hist, actions, levelNumNodes = result
            result = ( goal, hist, actions, numNodes + levelNumNodes )
            break
        else:
            levelNumNodes = result[0]
            numNodes = numNodes + levelNumNodes
    if ( result is None ):
        result = ( numNodes )
    return result
        

In [43]:
IDDFsearch( initial, 20 )

({'cabbage': 'right', 'farmer': 'right', 'sheep': 'right', 'wolf': 'right'},
 [{'cabbage': 'left', 'farmer': 'left', 'sheep': 'left', 'wolf': 'left'},
  {'cabbage': 'left', 'farmer': 'right', 'sheep': 'right', 'wolf': 'left'},
  {'cabbage': 'left', 'farmer': 'left', 'sheep': 'right', 'wolf': 'left'},
  {'cabbage': 'right', 'farmer': 'right', 'sheep': 'right', 'wolf': 'left'},
  {'cabbage': 'right', 'farmer': 'left', 'sheep': 'left', 'wolf': 'left'},
  {'cabbage': 'right', 'farmer': 'right', 'sheep': 'left', 'wolf': 'right'},
  {'cabbage': 'right', 'farmer': 'left', 'sheep': 'left', 'wolf': 'right'},
  {'cabbage': 'right', 'farmer': 'right', 'sheep': 'right', 'wolf': 'right'}],
 ['*** initial ***',
  'move sheep to right',
  'move farmer to left',
  'move cabbage to right',
  'move sheep to left',
  'move wolf to right',
  'move farmer to left',
  'move sheep to right',
  '*** goal ***'],
 66)